In [1]:
import pandas as pd
import numpy as np

In [2]:
# Name, Supply, Cost
suppliers = [('D1', 20, 10), ('D2', 30, 12)]

# Name, Demand, Price
receivers = [('O1', 10, 30), ('O2', 28, 25), ('O3', 27, 30)]

# rows:    suppliers
# columns: receivers
transport_cost_matrix = np.array([
    [8, 14, 17],
    [12, 9, 19]
])

In [3]:
def unit_profits_matrix(transport_cost_matrix, suppliers, receivers):
    un_pr_matrix = np.empty_like(transport_cost_matrix)
    # Suppliers
    for i, row in enumerate(transport_cost_matrix):
        # Receivers
        for j, num in enumerate(row):
            
            un_pr_matrix[i, j] = receivers[j][2] - suppliers[i][2] - transport_cost_matrix[i, j]
    
    return un_pr_matrix

def supply_demand_sum(suppliers, receivers, supp_idx=1, rec_idx=1):
    supp_sum = sum([supp[supp_idx] for supp in suppliers])
    dmnd_sum = sum([recv[rec_idx] for recv in receivers])
    return supp_sum, dmnd_sum

In [4]:
unit_profits = unit_profits_matrix(transport_cost_matrix, suppliers, receivers)

In [5]:
indices = np.argsort(unit_profits, axis=None)[::-1]
indices = np.unravel_index(indices, unit_profits.shape)
indices

(array([0, 1, 1, 0, 0, 1], dtype=int64),
 array([0, 0, 1, 2, 1, 2], dtype=int64))

In [6]:
supp_sum, dmnd_sum = supply_demand_sum(suppliers, receivers)

if supp_sum != dmnd_sum:
    unit_profits = np.pad(unit_profits, ((0, 1), (0, 1)), 'constant', constant_values=0)
    transport_cost_matrix = np.pad(transport_cost_matrix, ((0, 1), (0, 1)), 'constant', constant_values=0)
    suppliers.append(('DF', dmnd_sum, 0))
    receivers.append(('OF', supp_sum, 0))

In [7]:
unit_profits, suppliers, receivers

(array([[12,  1,  3,  0],
        [ 6,  4, -1,  0],
        [ 0,  0,  0,  0]]),
 [('D1', 20, 10), ('D2', 30, 12), ('DF', 65, 0)],
 [('O1', 10, 30), ('O2', 28, 25), ('O3', 27, 30), ('OF', 50, 0)])

In [8]:
def sales_matrix(unit_profits, suppliers, receivers, supp_idx=1, rec_idx=1):
    suppliers_cpy = [list(tup) for tup in suppliers]
    receivers_cpy = [list(tup) for tup in receivers]
    # We start from REAL Suppliers and Receivers:
    sales = np.zeros(unit_profits.shape)
    for idx, jdx in zip(*indices):
        # Each iteration is one supplier-receiver transaction
        unit_profit = unit_profits[idx, jdx]
        supp_name, supply, cost = suppliers_cpy[idx]
        recv_name, demand, price = receivers_cpy[jdx]
        if supply == 0 or demand == 0:
            continue
        
        supply_after = supply - demand
        if supply_after >= 0:
            # no supply
            sales[idx, jdx] = demand
            
            # Update
            suppliers_cpy[idx][supp_idx] = supply_after
            receivers_cpy[jdx][rec_idx]  = 0
        if supply_after < 0:
            # no demand
            sales[idx, jdx] = supply
            
            # Update
            suppliers_cpy[idx][supp_idx] = 0
            receivers_cpy[jdx][rec_idx]  = -supply_after
    
    # We go to FICTIONAL Suppliers and Receivers
    for idx, row in enumerate(sales):
        for jdx, num in enumerate(row):
            # Each iteration is one supplier-receiver transaction
            unit_profit = unit_profits[idx, jdx]
            supp_name, supply, cost = suppliers_cpy[idx]
            recv_name, demand, price = receivers_cpy[jdx]
            if supply == 0 or demand == 0:
                continue
            
            supply_after = supply - demand
            if supply_after >= 0:
                # no supply
                sales[idx, jdx] = demand
                
                # Update
                suppliers_cpy[idx][supp_idx] = supply_after
                receivers_cpy[jdx][rec_idx]  = 0
            if supply_after < 0:
                # no demand
                sales[idx, jdx] = supply
                
                # Update
                suppliers_cpy[idx][supp_idx] = 0
                receivers_cpy[jdx][rec_idx]  = -supply_after
            
    if (sum([supp[supp_idx] for supp in suppliers_cpy]) or
        sum([recv[rec_idx] for recv in receivers_cpy])):
        raise Exception("Supply and Demand after calculations are not zero.")
    return sales, suppliers_cpy, receivers_cpy

In [9]:
sales, supp, recv = sales_matrix(unit_profits, suppliers, receivers)

In [10]:
mask = sales[:-1, :-1] !=0
masked_array = (unit_profits[:-1, :-1] * mask)

A = []
B = []
for i in range(mask.shape[0]):
    for j in range(mask.shape[1]):
        if mask[i, j]:
            row = [0]*(mask.shape[0] + mask.shape[1])
            row[i] = 1
            row[mask.shape[0] + j] = 1
            A.append(row)
            B.append(masked_array[i, j])

A = np.array(A)
B = np.array(B)

solution, residuals, rank, s = np.linalg.lstsq(A, B, rcond=None)

solution

array([ 5.4,  1.4,  6.6,  2.6, -2.4])

In [11]:
# append 0 for fictional suppliers and receivers
alphas = np.append(solution[:2], 0)
betas = np.append(solution[2:], 0)

In [12]:
sales

array([[10.,  0., 10.,  0.],
       [ 0., 28.,  2.,  0.],
       [ 0.,  0., 15., 50.]])

In [13]:
def calculate_deltas(unit_profits, alphas, betas):
    # Step 5 in algorithm
    deltas = np.zeros(unit_profits.shape)
    for idx, row in enumerate(unit_profits[:-1][:-1]):
        for jdx, unit_profit in enumerate(row):
            deltas[idx, jdx] = round(unit_profit - alphas[idx] - betas[jdx], 4)
    
    return deltas

In [14]:
deltas = calculate_deltas(unit_profits, alphas, betas)

In [15]:
sales

array([[10.,  0., 10.,  0.],
       [ 0., 28.,  2.,  0.],
       [ 0.,  0., 15., 50.]])

In [16]:
unit_profits * sales

array([[120.,   0.,  30.,   0.],
       [  0., 112.,  -2.,   0.],
       [  0.,   0.,   0.,   0.]])

In [17]:
def calculate_overall_return(unit_profits, sales):
    return np.sum(unit_profits[:-1, :-1] * sales[:-1, :-1])

def calculate_returns(unit_profits, sales):
    return unit_profits[:-1, :-1] * sales[:-1, :-1]

In [18]:
calculate_overall_return(unit_profits, sales)

260.0

In [19]:
calculate_returns(unit_profits, sales)

array([[120.,   0.,  30.],
       [  0., 112.,  -2.]])

In [20]:
from util.middleman import MiddleMan
import numpy as np
mm = MiddleMan()
mm.add_supplier('D1', 20, 10)
mm.add_supplier('D2', 30, 12)

mm.add_receiver('O1', 10, 30)
mm.add_receiver('O2', 28, 25)
mm.add_receiver('O3', 27, 30)

transport_cost_matrix = np.array([
    [8, 14, 17],
    [12, 9, 19]
])

mm.add_transport_matrix(transport_cost_matrix)

mm.calculate()

len A, row A, B: (3, 5, 3)


'Optimal'

In [21]:
mm.calculate_overall_return()

262.0

In [22]:
mm.sales

array([[10.,  0., 10.,  0.],
       [ 0., 28.,  0.,  2.],
       [ 0.,  0., 17., 48.]])

In [23]:
mm._calculate_alpha_beta()

len A, row A, B: (3, 5, 3)


(array([5., 2., 0.]), array([ 7.,  2., -2.,  0.]), array([], dtype=float64))

In [24]:
mm.deltas

array([[ 0., -6.,  0., -5.],
       [-3.,  0., -1., -2.],
       [-7., -2.,  2.,  0.]])

In [25]:
mm.calculate_returns()

array([[120.,   0.,  30.],
       [  0., 112.,  -0.]])

In [26]:
import numpy as np

class MiddleMan:
    """
    # Example usage
    >>> mm = MiddleMan()
    >>> mm.add_supplier('D1', 20, 10)
    >>> mm.add_supplier('D2', 30, 12)

    >>> mm.add_receiver('O1', 10, 30)
    >>> mm.add_receiver('O2', 28, 25)
    >>> mm.add_receiver('O3', 27, 30)

    >>> transport_cost_matrix = np.array([
    >>>     [8, 14, 17],
    >>>     [12, 9, 19]
    >>> ])

    >>> mm.add_transport_matrix(transport_cost_matrix)

    >>> mm.calculate()
    'Optimal'
    >>> mm.calculate_overall_return()
    260.0
    >>> mm.calculate_returns()
    array([[120.,   0.,  30.],
           [  0., 112.,  -2.]])
    """
    def __init__(self, supp_idx=1, rec_idx=1):
        self.supp_idx = supp_idx
        self.rec_idx = rec_idx
        self.suppliers = []
        self.receivers = []
    
    def calculate(self):
        self.unit_profits = self.unit_profits_matrix()
        self.indices = self.calc_sorted_indices()
        
        supp_sum, dmnd_sum = self.supply_demand_sum()
        
        if supp_sum != dmnd_sum:
            print("Supply != Demand")
            self.unit_profits = np.pad(self.unit_profits, ((0, 1), (0, 1)), 'constant', constant_values=0)
            self.transport_cost_matrix = np.pad(self.transport_cost_matrix, ((0, 1), (0, 1)), 'constant', constant_values=0)
            self.suppliers.append(('DF', dmnd_sum, 0))
            self.receivers.append(('OF', supp_sum, 0))
        
        self.sales, supp, recv = self.sales_matrix()
        alphas, betas = self._calculate_alpha_beta()
        self.deltas = self.calculate_deltas(alphas, betas)
        
        if np.any(self.deltas[:-1, :-1] > 0):
            return "Suboptimal"
        else:
            return "Optimal"
        
    def add_supplier(self, name, supply, cost):
        self.suppliers.append((name, supply, cost))
        
    def add_receiver(self, name, demand, price):
        self.receivers.append((name, demand, price))
        
    def add_transport_matrix(self, matrix:np.ndarray):
        self.transport_cost_matrix = matrix
        
    def unit_profits_matrix(self):
        un_pr_matrix = np.empty_like(self.transport_cost_matrix)
        # Suppliers
        for i, row in enumerate(self.transport_cost_matrix):
            # Receivers
            for j, num in enumerate(row):
                
                un_pr_matrix[i, j] = self.receivers[j][2] - self.suppliers[i][2] - self.transport_cost_matrix[i, j]
        
        return un_pr_matrix

    def supply_demand_sum(self):
        supp_sum = sum([supp[self.supp_idx] for supp in self.suppliers])
        dmnd_sum = sum([recv[self.rec_idx] for recv in self.receivers])
        return supp_sum, dmnd_sum
    
    def calc_sorted_indices(self):
        self.indices = np.argsort(self.unit_profits, axis=None)[::-1]
        self.indices = np.unravel_index(self.indices, self.unit_profits.shape)
        return self.indices
    
    def sales_matrix(self):
        suppliers_cpy = [list(tup) for tup in self.suppliers]
        receivers_cpy = [list(tup) for tup in self.receivers]
        # We start from REAL Suppliers and Receivers:
        sales = np.zeros(self.unit_profits.shape)
        for idx, jdx in zip(*self.indices):
            # Each iteration is one supplier-receiver transaction
            unit_profit = self.unit_profits[idx, jdx]
            supp_name, supply, cost = suppliers_cpy[idx]
            recv_name, demand, price = receivers_cpy[jdx]
            
            # unit_profit < 0 -> middleman loses on this transaction.
            if supply == 0 or demand == 0 or unit_profit < 0:
                continue
            
            supply_after = supply - demand
            if supply_after >= 0:
                # no supply
                sales[idx, jdx] = demand
                
                # Update
                suppliers_cpy[idx][self.supp_idx] = supply_after
                receivers_cpy[jdx][self.rec_idx]  = 0
            if supply_after < 0:
                # no demand
                sales[idx, jdx] = supply
                
                # Update
                suppliers_cpy[idx][self.supp_idx] = 0
                receivers_cpy[jdx][self.rec_idx]  = -supply_after
        
        # We go to FICTIONAL Suppliers and Receivers
        for idx, row in enumerate(sales):
            for jdx, num in enumerate(row):
                # Each iteration is one supplier-receiver transaction
                unit_profit = self.unit_profits[idx, jdx]
                supp_name, supply, cost = suppliers_cpy[idx]
                recv_name, demand, price = receivers_cpy[jdx]
                if supply == 0 or demand == 0 or unit_profit < 0:
                    continue
                
                supply_after = supply - demand
                if supply_after >= 0:
                    # no supply
                    sales[idx, jdx] = demand
                    
                    # Update
                    suppliers_cpy[idx][self.supp_idx] = supply_after
                    receivers_cpy[jdx][self.rec_idx]  = 0
                if supply_after < 0:
                    # no demand
                    sales[idx, jdx] = supply
                    
                    # Update
                    suppliers_cpy[idx][self.supp_idx] = 0
                    receivers_cpy[jdx][self.rec_idx]  = -supply_after
                
        if (sum([supp[self.supp_idx] for supp in suppliers_cpy]) or
            sum([recv[self.rec_idx] for recv in receivers_cpy])):
            raise Exception("Supply and Demand after calculations are not zero.")
        return sales, suppliers_cpy, receivers_cpy
    
    def _calculate_alpha_beta(self):
        mask = self.sales[:-1, :-1] !=0
        masked_array = (self.unit_profits[:-1, :-1] * mask)
        
        A = []
        B = []
        for i in range(mask.shape[0]):
            for j in range(mask.shape[1]):
                if mask[i, j]:
                    row = [0]*(mask.shape[0] + mask.shape[1])
                    row[i] = 1
                    row[mask.shape[0] + j] = 1
                    A.append(row)
                    B.append(masked_array[i, j])

        A = np.array(A)
        B = np.array(B)
        
        solution, residuals, rank, s = np.linalg.lstsq(A, B, rcond=None)

        alphas = np.append(solution[:2], 0)
        betas = np.append(solution[2:], 0)
        
        return alphas, betas
    
    def calculate_deltas(self, alphas, betas):
        # Step 5 in algorithm
        deltas = np.zeros(self.unit_profits.shape)
        for idx, row in enumerate(self.unit_profits[:-1][:-1]):
            for jdx, unit_profit in enumerate(row):
                deltas[idx, jdx] = round(unit_profit - alphas[idx] - betas[jdx], 4)
        
        return deltas
    
    def calculate_overall_return(self):
        return np.sum(self.unit_profits[:-1, :-1] * self.sales[:-1, :-1])

    def calculate_returns(self):
        return self.unit_profits[:-1, :-1] * self.sales[:-1, :-1]

In [1]:
import numpy as np
from util.middleman import MiddleMan

mm = MiddleMan()
mm.add_supplier('D1', 20, 17)
mm.add_supplier('D2', 40, 7)

mm.add_receiver('O1', 10, 18)
mm.add_receiver('O2', 12, 16)
mm.add_receiver('O3', 24, 15)

transport_cost_matrix = np.array([
    [4, 7, 2],
    [8, 10, 4]
])

mm.add_transport_matrix(transport_cost_matrix)

mm.calculate()

'Optimal'

In [28]:
mm.unit_profits

array([[-3, -8, -4,  0],
       [ 3, -1,  4,  0],
       [ 0,  0,  0,  0]])

In [29]:
mm._calculate_alpha_beta()

len A, row A, B: (2, 5, 2)


(array([0.     , 2.33333, 0.     ]),
 array([0.66667, 0.     , 1.66667, 0.     ]),
 array([], dtype=float64))

In [30]:
mm.deltas

array([[-3.66667, -8.     , -5.66667,  0.     ],
       [-0.     , -3.33333, -0.     , -2.33333],
       [-0.66667,  0.     , -1.66667,  0.     ]])

In [31]:
mm.deltas[:-1, :-1]

array([[-3.66667, -8.     , -5.66667],
       [-0.     , -3.33333, -0.     ]])

In [32]:
mm.sales

array([[ 0.,  0.,  0., 20.],
       [10.,  0., 24.,  6.],
       [ 0., 12.,  0., 34.]])

In [33]:
mm.calculate_returns()

array([[-0., -0., -0.],
       [30., -0., 96.]])

In [34]:
mm.calculate_overall_return()

126.0

In [35]:
mm.sales==0

array([[ True,  True,  True, False],
       [False,  True, False, False],
       [ True, False,  True, False]])

# Przykładowe zadania

In [137]:
import numpy as np
# from util.middleman import MiddleMan

mm = MiddleMan()
mm.add_supplier('D1', 45, 6)
mm.add_supplier('D2', 25, 7)

mm.add_receiver('O1', 30, 12)
mm.add_receiver('O2', 30, 13)

transport_cost_matrix = np.array([
    [7, 4],
    [3, 5]
])

mm.add_transport_matrix(transport_cost_matrix)

mm.calculate()

[['D1', 45, 6], ['D2', 25, 7], ['DF', 60, 0]]
[['O1', 30, 12], ['O2', 30, 13], ['OF', 70, 0]]


'Optimal'

In [138]:
mm.unit_profits

array([[-1,  3,  0],
       [ 2,  1,  0],
       [ 0,  0,  0]])

In [139]:
print(mm.calculate_returns())
print("\nOverall return:\n")
print(mm.calculate_overall_return())

[[-5. 90.]
 [50.  0.]]

Overall return:

135.0


In [134]:
mm = MiddleMan()
mm.add_supplier('D1', 20, 7)
mm.add_supplier('D2', 40, 8)

mm.add_receiver('O1', 16, 18)
mm.add_receiver('O2', 12, 16)
mm.add_receiver('O3', 24, 15)

transport_cost_matrix = np.array([
    [4, 7, 2],
    [8, 10, 4]
])

mm.add_transport_matrix(transport_cost_matrix)

mm.calculate()

[['D1', 20, 7], ['D2', 40, 8], ['DF', 52, 0]]
[['O1', 16, 18], ['O2', 12, 16], ['O3', 24, 15], ['OF', 60, 0]]
[(0, 1), (0, 0), (1, 0), (1, 1)]
[[16.  0.  4.  0.]
 [ 0. 12. 20.  8.]
 [ 0.  0.  0. 52.]]
[16.0, 12.0]


'Optimal'

In [135]:
mm.unit_profits

array([[ 7,  2,  6,  0],
       [ 2, -2,  3,  0],
       [ 0,  0,  0,  0]])

In [136]:
print(mm.calculate_returns())
print("\nOverall return:\n")
print(mm.calculate_overall_return())

[[28. 24. 24.]
 [24. -0. 60.]]

Overall return:

160.0


# WORKING
#TODO: adding/removing suppliers/receivers and class calculating with no problem

In [399]:
class MiddleMan:
    def __init__(self, supp_idx=1, rec_idx=1, verbose=False):
        self.supp_idx = supp_idx
        self.rec_idx = rec_idx
        self.suppliers = []
        self.receivers = []
        self.verbose = verbose
        
    def __call__(self):
        return self.calculate()
    
    def calculate(self):
        """
        The most important function but I'm to lazy to describe it now, but it's rather obvious
        """
        if not hasattr(self, 'fictional'):
            self.unit_profits = self.unit_profits_matrix()
        self.indices = self.calc_sorted_indices()
        
        supp_sum, dmnd_sum = self.supply_demand_sum()
        
        if supp_sum != dmnd_sum:
            self.fictional = True
            self.unit_profits = np.pad(self.unit_profits, ((0, 1), (0, 1)), 'constant', constant_values=0)
            self.transport_cost_matrix = np.pad(self.transport_cost_matrix, ((0, 1), (0, 1)), 'constant', constant_values=0)
            self.suppliers.append(('DF', dmnd_sum, 0))
            self.receivers.append(('OF', supp_sum, 0))
        
        self.sales, supp, recv = self.sales_matrix()
        alphas, betas, residuals = self._calculate_alpha_beta()
        
        self.deltas = self.calculate_deltas(alphas, betas)
        iterator = 0
        while np.any(self.deltas > 0):
            if iterator == 1000:
                raise Exception("Infinite loop")
            iterator +=1
            # max_delta_idx = np.unravel_index(np.argmax(self.deltas[:-1, :-1]), self.deltas[:-1, :-1].shape)
            max_delta_idxs = self.calc_sorted_indices(self.deltas[:-1, :-1], inplace=False)
            
            for max_delta_idx in zip(*max_delta_idxs):
                if self.verbose:
                    print(f"STARTING AT {self.deltas[max_delta_idx]} delta value at {max_delta_idx}")
                if self.deltas[max_delta_idx] <=0:
                    break
                
                path = self._find_path(max_delta_idx) # max_delta_idx
                
                if path is not None:
                    self._update_sales_matrix(path)
                    break
                else:
                    if self.verbose:
                        print(f"PATH IS NONE FOR {self.deltas[max_delta_idx]} delta value") # max_delta_idx
                    continue                
                
            # if path is None:
            #     self.deltas[max_delta_idx] = -np.inf  # Mark as invalid
            #     continue
            
            # self._update_sales_matrix(path)
            if self.verbose:
                print("path:")
                print(path)
                print("deltas:")
                print(self.deltas)
            # print([mm.sales[idx] for idx in path if mm.sales[idx] > 0])
            
            alphas, betas, residuals = self._calculate_alpha_beta()
            self.deltas = self.calculate_deltas(alphas, betas)
            if self.verbose:
                print("new calculated deltas:")
                print(self.deltas)
        
        if self.verbose:
            print(iterator)
        
        if np.any(self.deltas[:-1, :-1] > 0):
            return "Suboptimal"
        else:
            return "Optimal"

    def _find_path(self, start_idx):
        """
        Responsible for finding a path in a deltas matrix e.g.:
        
        ```
        1--2
        |  |
        4--3
        ```
        
        Returns a path in order: `[1, 2, 3, 4]`
        
        1 is a starting point with delta > 0;  2, 3 and 4 have delta==0.
        """
        rows, cols = self.sales.shape
        start_row, start_col = start_idx
        
        for row in range(rows):
            if row != start_row:
                for col in range(cols):
                    if (col != start_col and row != start_row and 
                        self.sales[start_row, start_col] == 0 and self.sales[row, col] > 0 and
                        self.sales[start_row, col] > 0 and self.sales[row, start_col] > 0):
                        
                        return [(start_row, start_col), (start_row, col), (row, col), (row, start_col)]
        
        return None

    def _update_sales_matrix(self, path):
        """
        Changes the sales matrix with given path - a list of tuples - positions.
        
        Takes the minimum of second and third element of path (corners of path):
        
        e.g.:
        
        ```
        1--2
        |  |
        4--3
        ```
        """
        min_sales_value = min(self.sales[path[1]], self.sales[path[3]])
        # min_sales_value = min(self.sales[idx] for idx in path if self.sales[idx] > 0)

        for i, idx in enumerate(path):
            if i % 2 == 0:
                self.sales[idx] += min_sales_value
            else:
                self.sales[idx] -= min_sales_value

    
    def add_supplier(self, name, supply, cost):
        self.suppliers.append((name, supply, cost))
        
    def add_receiver(self, name, demand, price):
        self.receivers.append((name, demand, price))
        
    def add_transport_matrix(self, matrix:np.ndarray):
        self.transport_cost_matrix = matrix
        
    def unit_profits_matrix(self):
        """
        Calculates unit profits from suppliers and receivers
        """
        un_pr_matrix = np.empty_like(self.transport_cost_matrix)
        # Suppliers
        for i, row in enumerate(self.transport_cost_matrix):
            # Receivers
            for j, num in enumerate(row):
                un_pr_matrix[i, j] = self.receivers[j][2] - self.suppliers[i][2] - self.transport_cost_matrix[i, j]
        
        return un_pr_matrix

    def supply_demand_sum(self):
        supp_sum = sum([supp[self.supp_idx] for supp in self.suppliers])
        dmnd_sum = sum([recv[self.rec_idx] for recv in self.receivers])
        return supp_sum, dmnd_sum
    
    def calc_sorted_indices(self, matrix=None, inplace=True):
        """
        Sorts elements and returns their position in array.
        """
        if matrix is None:
            if hasattr(self, 'fictional'):
                matrix = self.unit_profits[:-1, :-1]
            else:
                matrix = self.unit_profits
                
        
        indices = np.argsort(matrix, axis=None)[::-1]
        indices = np.unravel_index(indices, matrix.shape)
        
        if inplace:
            self.indices = indices
        
        return indices
    
    def sales_matrix(self):
        """
        Function for calculating the initial sales matrix.
        
        Staring from real suppliers and receivers:
        - iterating through matrix from highest to lowest unit incomes
        - assigning to each matrix a highest sale according to a supply / demand
        
        
        """
        suppliers_cpy = [list(tup) for tup in self.suppliers]
        receivers_cpy = [list(tup) for tup in self.receivers]
        
        # We start from REAL Suppliers and Receivers:
        sales = np.zeros(self.unit_profits.shape)
        for idx, jdx in zip(*self.indices):
            # Each iteration is one supplier-receiver transaction
            unit_profit = self.unit_profits[idx, jdx]
            supp_name, supply, cost = suppliers_cpy[idx]
            recv_name, demand, price = receivers_cpy[jdx]
            
            # unit_profit < 0 -> middleman loses on this transaction.
            if supply == 0 or demand == 0: #or unit_profit < 0:
                continue
            
            supply_after = supply - demand
            if supply_after >= 0:
                # no supply
                sales[idx, jdx] = demand
                
                # Update
                suppliers_cpy[idx][self.supp_idx] = supply_after
                receivers_cpy[jdx][self.rec_idx]  = 0
            if supply_after < 0:
                # no demand
                sales[idx, jdx] = supply
                
                # Update
                suppliers_cpy[idx][self.supp_idx] = 0
                receivers_cpy[jdx][self.rec_idx]  = -supply_after
        
        if hasattr(self, 'fictional'):
            # We go to FICTIONAL Suppliers and Receivers
            for idx, row in enumerate(sales):
                for jdx, num in enumerate(row):
                    # Each iteration is one supplier-receiver transaction
                    unit_profit = self.unit_profits[idx, jdx]
                    supp_name, supply, cost = suppliers_cpy[idx]
                    recv_name, demand, price = receivers_cpy[jdx]
                    if supply == 0 or demand == 0: #or unit_profit < 0:
                        continue
                    
                    supply_after = supply - demand
                    if supply_after >= 0:
                        # no supply
                        sales[idx, jdx] = demand
                        
                        # Update
                        suppliers_cpy[idx][self.supp_idx] = supply_after
                        receivers_cpy[jdx][self.rec_idx]  = 0
                    if supply_after < 0:
                        # no demand
                        sales[idx, jdx] = supply
                        
                        # Update
                        suppliers_cpy[idx][self.supp_idx] = 0
                        receivers_cpy[jdx][self.rec_idx]  = -supply_after
                
        if (sum([supp[self.supp_idx] for supp in suppliers_cpy]) or
            sum([recv[self.rec_idx] for recv in receivers_cpy])):
            raise Exception("Supply and Demand after calculations are not zero.")
        return sales, suppliers_cpy, receivers_cpy
    
    def _calculate_alpha_beta(self):
        """
        Function that calculates alpha and beta coefficients by using a system of equations.
        
        Takes non-zero `sales` array elements and creates for them the alpha_i and beta_j equations.
        """
        mask = self.sales != 0
        masked_array = (self.unit_profits * mask)
        
        if self.verbose:
            print("sales:")
            print(self.sales)
            print("unit_profits on mask:")
            print(masked_array)
        
        A = []
        B = []
        for i in range(mask.shape[0]):
            for j in range(mask.shape[1]):
                if mask[i, j]:
                    row = [0]*(mask.shape[0] + mask.shape[1])
                    row[i] = 1
                    row[mask.shape[0] + j] = 1
                    A.append(row)
                    B.append(masked_array[i, j])
    
        num_variables = mask.shape[0] + mask.shape[1]
        num_equations = len(A)
    
        # If there are more variables than equations, add equations to make first variables equal to zero
        if num_variables > num_equations:
            for i in range(num_variables - num_equations):
                row = [0]*num_variables
                row[i] = 1
                A.append(row)
                B.append(0)
        else:
            if self.verbose:
                print("NO ADDITIONAL VARIABLE")
        
        # Alternatively we can set alpha 0 to be equal to 0
        # row = [0]*num_variables
        # row[i] = 1
        # A.append(row)
        # B.append(0)
                
        A = np.array(A)
        B = np.array(B)
    
        solution, residuals, rank, s = np.linalg.lstsq(A, B, rcond=None)
        if len(residuals)!=0:
            raise Exception(f"System of equations has found solution with residuals = {residuals}")
        
        alphas = solution[:mask.shape[0]].round(5)#, 0)
        betas = solution[mask.shape[0]:].round(5)#, 0)
    
        return alphas, betas, residuals
    
    def calculate_deltas(self, alphas, betas):
        """
        Creates a matrix of deltas - a determinant of optimum. If any element is positive, 
        than the result is not optimal.
        
        Each element equation is like so: delta_i_j = z_i_j - alpha_i - beta_j
        
        where z_i_j is a unit profit on the transit and alpha and beta are according 
        coefficients calculated earlier.
        """
        # Step 5 in algorithm
        
        deltas = np.zeros(self.unit_profits.shape)
        for idx, row in enumerate(self.unit_profits):
            for jdx, unit_profit in enumerate(row):
                deltas[idx, jdx] = round(unit_profit - alphas[idx] - betas[jdx], 5)
        
        return deltas
    
    def calculate_overall_return(self):
        return np.sum(self.unit_profits[:-1, :-1] * self.sales[:-1, :-1])

    def calculate_returns(self):
        return self.unit_profits[:-1, :-1] * self.sales[:-1, :-1]

In [400]:
import numpy as np
# from util.middleman import MiddleMan

mm = MiddleMan()
mm.add_supplier('D1', 20, 10)
mm.add_supplier('D2', 30, 12)
mm.add_supplier('D3', 55, 14)

mm.add_receiver('O1', 28, 30)
mm.add_receiver('O2', 37, 30)
mm.add_receiver('O3', 45, 25)

transport_cost_matrix = np.array([
    [17, 15, 6],
    [7, 7, 1],
    [15, 14, 3]
])

mm.add_transport_matrix(transport_cost_matrix)

mm.calculate()

'Optimal'

In [401]:
mm.deltas

array([[ -2.,   0.,  -2.,  -5.],
       [  0.,   0.,  -5., -11.],
       [ -1.,   0.,   0.,  -2.],
       [  0.,   0.,  -6.,   0.]])

In [402]:
mm.calculate()

'Optimal'

In [403]:
mm()

'Optimal'

In [404]:
mm._calculate_alpha_beta()

(array([ 0.,  6., -3., -5.]),
 array([ 5.,  5., 11.,  5.]),
 array([], dtype=float64))

In [405]:
mm.deltas

array([[ -2.,   0.,  -2.,  -5.],
       [  0.,   0.,  -5., -11.],
       [ -1.,   0.,   0.,  -2.],
       [  0.,   0.,  -6.,   0.]])

In [406]:
mm.unit_profits[:-1, :-1]

array([[ 3,  5,  9],
       [11, 11, 12],
       [ 1,  2,  8]])

In [407]:
mm.sales

array([[  0.,  20.,   0.,   0.],
       [ 23.,   7.,   0.,   0.],
       [  0.,  10.,  45.,   0.],
       [  5.,   0.,   0., 105.]])

In [408]:
mm.calculate_overall_return()

810.0

In [45]:
sample_arr = np.array([[  0.,   5.,  15.,   0.],
                       [  1.,   1.,  28.,   0.],
                       [ 22.,  31.,   2.,   0.],
                       [  5.,   0.,   0., 105.]]) # [:-1, :-1] * mm.unit_profits[:-1, :-1])


mask = sample_arr[:-1, :-1] !=0
masked_array = (mm.unit_profits[:-1, :-1] * mask)

A = []
B = []
for i in range(mask.shape[0]):
    for j in range(mask.shape[1]):
        if mask[i, j]:
            row = [0]*(mask.shape[0] + mask.shape[1])
            row[i] = 1
            row[mask.shape[0] + j] = 1
            A.append(row)
            B.append(masked_array[i, j])

A = np.array(A)
B = np.array(B)

solution, residuals, rank, s = np.linalg.lstsq(A, B, rcond=None)
                
alphas = np.append(solution[:mask.shape[0]].round(5), 0)
betas = np.append(solution[mask.shape[0]:].round(5), 0)

deltas = np.zeros(mm.unit_profits.shape)
for idx, row in enumerate(mm.unit_profits):
    for jdx, unit_profit in enumerate(row):
        deltas[idx, jdx] = round(unit_profit - alphas[idx] - betas[jdx], 5)

print(alphas)
print(betas)
print(deltas)

[2.70833 7.79167 0.125   0.     ]
[2.04167 2.45833 6.125   0.     ]
[[-1.75    -0.16666  0.16667 -2.70833]
 [ 1.16666  0.75    -1.91667 -7.79167]
 [-1.16667 -0.58333  1.75    -0.125  ]
 [-2.04167 -2.45833 -6.125    0.     ]]


In [46]:
mm.deltas

array([[-1.,  0.,  0., -3.],
       [ 4.,  3.,  0., -6.],
       [ 0.,  0.,  2.,  0.],
       [-1., -2., -6.,  0.]])

In [47]:
mm._calculate_alpha_beta()

len A, row A, B: (5, 6, 5)


(array([3., 6., 0., 0.]), array([1., 2., 6., 0.]), array([], dtype=float64))

In [48]:
print(mm.calculate_returns())
print("\nOverall return:\n")
print(mm.calculate_overall_return())

[[  0.  25. 135.]
 [  0.   0. 360.]
 [ 23.  64.   0.]]

Overall return:

607.0
